## 빅데이터 실습

### 스타벅스매장 입지 분석

#### 데이터 수집

##### 데이터 전처리 마무리

In [1]:
# 필요 라이브러리
import pandas as pd

In [2]:
# 서울시 스타벅스 매장정보 로드
dfStarbucksSeoul = pd.read_excel('./data/starbucks_seoul_list.xlsx')
dfStarbucksSeoul

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.513931,127.020606,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩,37.494668,127.062583,general,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
...,...,...,...,...,...,...
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232
609,상봉역,37.596890,127.086470,general,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232
611,양원역,37.606654,127.106360,general,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232


In [26]:
# 주소를 ''로 자른다음(배열) 1번째 인덱스 값이 '구'
guName = []
for address in dfStarbucksSeoul['주소']:
    addr = address.split(' ')
    guName.append(addr[1])
dfStarbucksSeoul['구'] = guName


In [7]:
dfStarbucksSeoul.tail()

,매장명,위도,경도,매장타입,주소,전화번호,구
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232,중랑구
609,상봉역,37.596890,127.086470,general,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,중랑구
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,중랑구
611,양원역,37.606654,127.106360,general,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,중랑구
612,중화역,37.601709,127.078411,general,서울특별시 중랑구 봉화산로 35 1층,1522-3232,중랑구


In [8]:
dfStarbucksSeoul.to_excel('./data/최종_서울_스타벅스매장리스트.xlsx', index=False)

In [10]:
# 스타벅스 분석 데이터 만들기
dfSeoulGu = pd.read_excel('./data/수정_서울_시군구_위경도.xlsx')
dfSeoulGu.tail()

,docity,do,city,longitude,latitude
20,서울용산구,서울,용산구,126.967522,37.536094
21,서울은평구,서울,은평구,126.931242,37.599969
22,서울종로구,서울,종로구,126.981642,37.570378
23,서울중구,서울,중구,126.999642,37.561003
24,서울중랑구,서울,중랑구,127.094778,37.603806


In [13]:
# docity 컬럼 삭제, 위경도 위치 변경
dfSeoulGuLast = dfSeoulGu[['do','city','latitude','longitude']]

In [14]:
# 컬럼 한국어로 변경
dfSeoulGuLast.columns = ['시','구','위도','경도']

In [16]:
dfSeoulGuLast.tail()

,시,구,위도,경도
20,서울,용산구,37.536094,126.967522
21,서울,은평구,37.599969,126.931242
22,서울,종로구,37.570378,126.981642
23,서울,중구,37.561003,126.999642
24,서울,중랑구,37.603806,127.094778


In [18]:
dfSeoulGuLast.to_excel('./data/최종_서울시_구_리스트.xlsx', index=False)

In [21]:
## 서울시 스타벅스 매장리스트에서 구별로 데이터 합산(pivot_table)
starbucksSeoulCount = dfStarbucksSeoul.pivot_table(values='매장명', index='구',
                             aggfunc='count').rename(columns={'매장명':'매장수'})
starbucksSeoulCount.tail()

,매장수
구,
용산구,24
은평구,14
종로구,40
중구,54
중랑구,8


In [24]:
# 서울구 리스트에 스타벅스별 매장수 병합, 최종데이터 > last를 앞에
lastSbSeoulCount = pd.merge(left=dfSeoulGuLast, right = starbucksSeoulCount, how='left',on='구')

In [33]:
# 서울시 구별 인구통계 로드
dfSeoulGuPop = pd.read_excel('./data/수정_서울_구별인구.xlsx')
dfSeoulGuPop

,구,세대소계,남자,여자,한국인소계,남자.1,여자.1,등록외국인소계,남자.2,여자.2,세대당인구,65세이상
0,4469417,9638799,4649446,4989353,9386034,4540031,4846003,252765,109415,143350,2.10,1743696
1,72067,150453,71890,78563,139417,67306,72111,11036,4584,6452,1.93,28764
2,64714,131793,63495,68298,121312,58659,62653,10481,4836,5645,1.87,25920
3,107825,227106,109826,117280,213151,102312,110839,13955,7514,6441,1.98,39747
4,133089,284766,137620,147146,277361,134519,142842,7405,3101,4304,2.08,49462
5,170077,351180,167562,183618,335554,161277,174277,15626,6285,9341,1.97,56819
6,172801,359873,174120,185753,341149,167346,173803,18724,6774,11950,1.97,67030
7,188097,387470,189462,198008,382155,187372,194783,5315,2090,3225,2.03,78955
8,196800,438168,208682,229486,425602,204171,221431,12566,4511,8055,2.16,80414
9,143560,292977,141185,151792,288113,139514,148599,4864,1671,3193,2.01,68617


In [38]:
dfSeoulGuPop[['구','성별소계']]

KeyError: "['성별소계'] not in index"

In [30]:
lastSbSeoulCount = pd.merge(left=lastSbSeoulCount, right=dfSeoulGuPop[['구','성별소계']], how = 'left', on= '구')

KeyError: "['성별소계'] not in index"

In [ ]:
lastSbSeoulCount.tail()

In [37]:
# 사업체 정보에 들어올 종사자수 컬럼이름이 '성별소계'로 동일, 컬럼이름 변경
lastSbSeoulCount.columns = ['시','구','위도','경도','스타벅스매장수','주민등록인구']

ValueError: Length mismatch: Expected axis has 5 elements, new values have 6 elements

In [36]:
# 사업체 정보 로드
dfSeoulGubiz = pd.read_excel('./data/수정_서울_구별사업체.xlsx')
dfSeoulGubiz.tail()

,구,소계,여성대표자,성별소계,남자,여자
21,관악구,37915,13641,128417,68392,60025
22,서초구,71958,23066,487976,290084,197892
23,강남구,110007,35453,801419,447809,353610
24,송파구,74531,25400,400781,244588,156193
25,강동구,41660,15226,157465,83415,74050


In [39]:
# 사업체정보에서 컬럼이름부터 변경
dfSeoulGubiz = dfSeoulGubiz[['구','소계','성별소계']]
dfSeoulGubiz.columns = ['구' , '사업체수', '종사자수']

In [ ]:
# 사업체 정보를 스타벅스 최종데이터에 병합
lastSbSeoulCount = pd.merge(left=lastSbSeoulCount , right = dfSeoulGubiz , how='left', on='구')

In [ ]:
lastSbSeoulCount.to_excel('./data/최종_서울_스타벅스매장_데이터.xlsx', index=False)

##### 시각화

In [51]:
# 시각화 라이브러리
import folium
import json
from folium.features import DivIcon

In [3]:
# 새로 엑셀에서 서울시 스타벅스 매장위치 로드
dfStarbucksSeoul = pd.read_excel('./data/최종_서울_스타벅스매장리스트.xlsx')

In [4]:
# 엑셀불러오기
dfStarbucksSeoulCount = pd.read_excel('./data/최종_서울_스타벅스매장_데이터.xlsx')

In [5]:
# 서울시 지도 생성
seoulLoc = [37.550800, 126.989857]
seoulMap = folium.Map(location=seoulLoc, zoom_start = 11)

# 각 매장별 위치를 지도에 마커로 표시
for idx in dfStarbucksSeoul.index:
    lat = dfStarbucksSeoul.loc[idx,'위도']
    lng = dfStarbucksSeoul.loc[idx,'경도']
    # 서클마커
    folium.CircleMarker(location=[lat,lng], fill=True, fill_color='white', fill_opacity=0.5, 
                        color='blue', weight=1,radius=3).add_to(seoulMap)

seoulMap

In [6]:
# 매장타입 갯수
dfStarbucksSeoul['매장타입'].value_counts()

매장타입
general      552
reserve       36
generalDT     23
generalWT      2
Name: count, dtype: int64

In [7]:
# 서울시 지도 생성
seoulLoc = [37.550800, 126.989857]
sbMap = folium.Map(location=seoulLoc, zoom_start = 11)

# 각 매장별 위치를 지도에 마커로 표시
for idx in dfStarbucksSeoul.index:
    lat = dfStarbucksSeoul.loc[idx,'위도']
    lng = dfStarbucksSeoul.loc[idx,'경도']
    storeType = dfStarbucksSeoul.loc[idx,'매장타입']
    storeName = dfStarbucksSeoul.loc[idx,'매장명']

    # 매장타입별 색상
    fillColor = ''
    if storeType == 'general':
        fillColor = 'darkgrean'; size = 2
    elif storeType == 'reserve':
        fillColor = 'skyblue'; size = 5
    elif storeType == 'generalDT':
        fillColor = 'purple'; size = 4
    elif storeType == 'generalWT':
        fillColor = 'white'; size = 3


    # 서클마커
    folium.CircleMarker(location=[lat,lng], popup=storeName, fill=True, fill_color=fillColor, fill_opacity=0.5, 
                        color='blue', weight=1,radius=size).add_to(sbMap)

sbMap

In [8]:
# 서울시 구별 스타벅스 매장수 시각화
dfStarbucksSeoulCount = pd.read_excel('./data/최종_서울_스타벅스매장_데이터.xlsx', thousands=',')
dfStarbucksSeoulCount.tail()

,시,구,위도,경도,스타벅스매장수,주민등록인구,사업체수,종사자수
20,서울,용산구,37.536094,126.967522,24,227106,29680,152605
21,서울,은평구,37.599969,126.931242,14,470869,36509,109031
22,서울,종로구,37.570378,126.981642,40,150453,48361,275063
23,서울,중구,37.561003,126.999642,54,131793,70308,386564
24,서울,중랑구,37.603806,127.094778,8,387470,39310,113144


In [9]:
# 서울시 행정구역(구별) 경계지도 파일
path = './data/seoul_municipalities_geo_simple.json'
seoulGeoSimple = json.load(open(path, encoding='utf-8'))

In [10]:
seoulGeoSimple['features'][0]['properties']

{'code': '11250',
 'name': '강동구',
 'name_eng': 'Gangdong-gu',
 'base_year': '2013'}

In [21]:
def style_function(feature):
    return{
        'opacity':0.7,'weight':1,'color':'red','fillOpacity':0,'dashArray':'5,5'
    }

In [23]:
## 지도생성
sbCountMap = folium.Map(location=seoulLoc, zoom_start=11, tiles = 'cartoDB.dark_matter')

folium.GeoJson(seoulGeoSimple, style_function=style_function).add_to(sbCountMap)

sbCountMap

In [14]:
## 스타벅스 평균 매장 수 계산
sbMean = dfStarbucksSeoulCount['스타벅스매장수'].mean()

In [42]:
## 지도생성
sbCountMap = folium.Map(location=seoulLoc, zoom_start=11, tiles = 'cartoDB.dark_matter')

folium.GeoJson(seoulGeoSimple, style_function=style_function).add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx, '위도']
    lng = dfStarbucksSeoulCount.loc[idx, '경도']
    count = dfStarbucksSeoulCount.loc[idx, '스타벅스매장수']
    guName = dfStarbucksSeoulCount.loc[idx, '구']
    if count > sbMean:
        fillColor = 'red' # 평균보다 크면 빨강색
    else: 
        fillColor= 'blue'
    folium.CircleMarker(location=[lat,lng], popup=guName+',\n'+str(count), color='FFFF00', fill_color=fillColor, 
                        fill_opacity=0.5, weight=1.5, radius=count/2).add_to(sbCountMap)


sbCountMap

In [ ]:
dfStarbucksSeoulCount.loc[8,'위도']= 
dfStarbucksSeoulCount.loc[8,'경도']=


In [55]:
## 구별 스타벅스 매장수 시각화
sbCountMap = folium.Map(location=seoulLoc, zoom_start=11)

folium.GeoJson(seoulGeoSimple, style_function=style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple, data=dfStarbucksSeoulCount, columns=['구','스타벅스매장수'],
                  fill_color='YlGn',fill_opacity=0.7, line_opacity=0.5, key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx, '위도']
    lng = dfStarbucksSeoulCount.loc[idx, '경도']
    count = dfStarbucksSeoulCount.loc[idx, '스타벅스매장수']
    guName = dfStarbucksSeoulCount.loc[idx, '구']
    # 마커와 DivIcon으로 지도에 텍스트 표시
    folium.Marker(location=[lat,lng], icon=DivIcon(
        icon_size=(150,60),
        icon_anchor=(75,25),
        html=f'''
        <div style="font-size: 9pt; font-weight: bold, color: black; text-align: center;">
            {guName} / {count}
        </div>
        '''
    )).add_to(sbCountMap)
sbCountMap

In [62]:
## 시울시 구별 인구수 시각화
sbCountMap = folium.Map(location=seoulLoc, zoom_start=11)

folium.GeoJson(seoulGeoSimple, style_function=style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple, data=dfStarbucksSeoulCount, columns=['구','스타벅스매장수'], 
                  legend_name='구별 인구분포',fill_color='Blues',fill_opacity=0.7, line_opacity=0.5,
                  key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx, '위도']
    lng = dfStarbucksSeoulCount.loc[idx, '경도']
    count = dfStarbucksSeoulCount.loc[idx, '주민등록인구']
    guName = dfStarbucksSeoulCount.loc[idx, '구']
    # 마커와 DivIcon으로 지도에 텍스트 표시
    folium.Marker(location=[lat,lng], icon=DivIcon(
        icon_size=(150,60),
        icon_anchor=(75,25),
        html=f'''
        <div style="font-size: 9pt; font-weight: bold, color: black; text-align: center;">
            {guName} / {count}
        </div>
        '''
    )).add_to(sbCountMap)
sbCountMap

In [67]:
## 서울시 구별 사업체수 지도시각화
sbCountMap = folium.Map(location=seoulLoc, zoom_start=11)

folium.GeoJson(seoulGeoSimple, style_function=style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple, data=dfStarbucksSeoulCount, columns=['구','사업체수'], 
                  legend_name='구별 사업체분포',fill_color='YlOrRd',fill_opacity=0.7, line_opacity=0.5,
                  key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx, '위도']
    lng = dfStarbucksSeoulCount.loc[idx, '경도']
    count = dfStarbucksSeoulCount.loc[idx, '사업체수']
    guName = dfStarbucksSeoulCount.loc[idx, '구']
    # 마커와 DivIcon으로 지도에 텍스트 표시
    folium.Marker(location=[lat,lng], icon=DivIcon(
        icon_size=(150,60),
        icon_anchor=(75,25),
        html=f'''
        <div style="font-size: 9pt; font-weight: bold, color: black; text-align: center;">
            {guName} / {count}
        </div>
        '''
    )).add_to(sbCountMap)
sbCountMap

##### 결론 
- 스타벅스는 인구수와 별개로 직장인이 많이 분포하는 구에 매장을 많이 오픈함